# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
#from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv('weather.csv')
weather_df.head()

,Unnamed: 0,City,Country,Temp,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed
0,0,chokurdakh,RU,25.83,70.63,147.92,25.83,46,73,1.53
1,1,butaritari,KI,28.52,3.07,172.79,28.52,75,26,6.70
2,2,anloga,GH,24.00,5.79,0.90,24.00,88,20,4.60
3,3,ponta do sol,PT,20.17,32.67,-17.10,22.00,68,20,6.20
4,4,bredasdorp,ZA,13.00,-34.53,20.04,13.00,93,0,1.00


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_df[["Latitude", "Longitude"]].astype(float)

fig = gmaps.figure()
humidity =weather_df['Humidity']
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 2)

fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
ideal_temp_df = weather_df.loc[weather_df['Temp'] >= 21]
ideal_temp_df = ideal_temp_df.loc[ideal_temp_df['Temp'] <= 26.5]
ideal_wind_df = ideal_temp_df.loc[ideal_temp_df['Wind Speed']<=16]
ideal_cloud_df = ideal_wind_df.loc[ideal_wind_df['Cloudiness']==0]
ideal_df = ideal_cloud_df[['City','Country','Temp','Latitude','Longitude','Cloudiness','Wind Speed']]
print(ideal_df.head())

             City Country   Temp  Latitude  Longitude  Cloudiness  Wind Speed
8      bani walid      LY  22.31     31.76      13.99           0        4.59
54         atuona      PF  26.49     -9.80    -139.03           0        7.93
128  palaikastron      GR  25.56     35.20      26.25           0        8.49
135       cascais      PT  22.31     38.70      -9.42           0        6.70
136       albania      AL  22.00     41.00      20.00           0        1.00


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [20]:
hotel_df = ideal_df
#print(hotel_df)
hotel_df['Hotel Name'] = ""


params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

for i, j in hotel_df.iterrows():
    lat = j['Latitude']
    lng = j['Longitude']
    params['location'] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params).json()

    try:
        hotel_df.loc[i, "Hotel Name"] = response['results'][0]['name']
    except (IndexError, KeyError):
        print('Hotel not found')



C:\Users\tonyg\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\tonyg\anaconda3\lib\site-packages\pandas\core\indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


Hotel not found
Hotel not found
Hotel not found


In [21]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [26]:

marker_locations = hotel_df[['Latitude', 'Longitude']]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations, info_box_content = [f'Hotel:{hotel}' for hotel in hotel_info])
                                                                   
fig.add_layer(markers)
fig
#info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate]
#**Hotel Name**, **City**, and **Country**

Figure(layout=FigureLayout(height='420px'))

In [24]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

# Display figure


Figure(layout=FigureLayout(height='420px'))